In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf

%matplotlib inline
#%matplotlib tk

In [2]:
alpha1 = 1.676
beta1sr = 0.18219 
capLam1 = .504
rho1 = -.964
sigma10 = .16376

alpha2 = 6.488
beta2sr = .21531
capLam2 = 2.115
rho2 =  -1
sigma20 = .07984


lam =  .064
meanCo =  .065
rhoJ =  -3.304

norMu = -.280 - rhoJ*meanCo
norStd = (.413**2 - (rhoJ*meanCo)**2)**.5

lamPri = .013
meanId = .052

alpha = np.array([[np.nan, alpha1, alpha2]]).T
betaSr = np.array([[np.nan, beta1sr, beta2sr]]).T
capLam = np.array([[np.nan, capLam1, capLam2]]).T
rho = np.array([[np.nan, rho1, rho2]]).T
sigma0 = np.array([[np.nan, sigma10, sigma20]]).T


In [3]:
def xi(z):
    
    xi = alpha - rho * capLam @ (1j *z)
    return xi

In [4]:
def dk(z):
    dk = np.sqrt(xi(z) ** 2 + z * (1j + z) * capLam ** 2)
    return dk

In [5]:
def GcoPlus(z):
    GcoPlus = 1 - 1j * z * rhoJ * meanCo - (meanCo / (capLam[1] ** 2)) * (xi(z)[1] + dk(z)[1])
    return GcoPlus

def GcoMinus(z):
    GcoMinus = 1 - 1j * z * rhoJ * meanCo - (meanCo / (capLam[1] ** 2)) * (xi(z)[1] - dk(z)[1])
    return GcoMinus


In [6]:
def GidPlus(z):
    GidPlus = 1 - (meanId / (capLam[1] ** 2)) * (xi(z)[1] + dk(z)[1])
    return GidPlus

def GidMinus(z):
    GidMinus = 1 - (meanId / (capLam[1] ** 2)) * (xi(z)[1] - dk(z)[1])
    return GidMinus


In [7]:
def gk(z):
    gk = (xi(z) - dk(z)) / (xi(z) + dk(z))
    return gk

In [8]:
def AkLogPrice(z, tau ):
    Ak = (alpha*betaSr**2/(capLam**2))*((xi(z)-dk(z))*tau-2*np.log((1-gk(z)*np.exp(-dk(z)*tau))/(1-gk(z))))
    return Ak

In [9]:
def BkLogPrice(z, tau):
    Bk = ((xi(z) - dk(z))*(1 - np.exp(-dk(z)*tau))) / (capLam**2*(1-gk(z)*np.exp(-dk(z)*tau)))
    return Bk

In [10]:
def ThetaCo(z, tau):
    theta1 = np.exp(1j*norMu*z - .5*norStd**2*z**2)
    logBlob = np.log((GcoMinus(z)-gk(z)[1]*GcoPlus(z)*np.exp(-dk(z)[1]*tau))/((1-gk(z)[1])*(1-1j*z*rhoJ*meanCo)))
    theta2 = (1/GcoMinus(z))*(1-(2*meanCo*logBlob)/(tau*capLam[1]**2*GcoPlus(z)))
    thetaCo = theta1 * theta2
    return thetaCo

def capCcoPrice(z, tau):
    muBar = np.exp(norMu+.5*norStd**2)/(1-meanCo*rhoJ) - 1
    capC = lam*tau*(ThetaCo(z, tau)-1-1j*muBar*z)
    return capC

In [11]:
def ThetaId(z, tau):
    logBlob = np.log((GidMinus(z)-gk(z)[1]*GidPlus(z)*np.exp(-dk(z)[1]*tau))/(1-gk(z)[1]))
    thetaId = (1/GidMinus(z))*(1-(2*meanId*logBlob)/(tau*capLam[1]**2*GidPlus(z)))
    return thetaId

def capCidPrice(z, tau):
    capC = lamPri*tau*(ThetaId(z, tau)-1)
    return capC

In [12]:
def AkSV(z, tau):
    Ak = -2*(alpha*betaSr**2/(capLam**2))*np.log(1-(capLam**2/(2*alpha)*(1-np.exp(-alpha*tau)))@(1j*z))
    return Ak

In [13]:
def BkSV(z, tau):
    Bk = (np.exp(-alpha*tau)@(1j*z))/(1-(capLam**2/(2*alpha)*(1-np.exp(-alpha*tau)))@(1j*z))
    return Bk

In [14]:
def Theta(z, mu, tau):
    logBlob = np.log(1-(1j*z*(capLam[1]**2-2*alpha[1]*mu)*(1-np.exp(-alpha[1]*tau)))/(2*alpha[1]*(1-1j*z*mu)))
    theta = -2*mu/(capLam[1]**2-2*alpha[1]*mu) * logBlob
    return theta

def capCcoSV(z, tau):
    capC = lam*Theta(z, meanCo, tau)
    return capC

def capCidSV(z, tau):
    capC = lamPri*Theta(z, meanId, tau)
    return capC

In [15]:
tauBar = 30/365
def aK(tauBar):
    ak = (1-np.exp(-tauBar*alpha))/alpha
    return ak

def b1(tauBar):
    muBar = np.exp(norMu+.5*norStd**2)/(1-meanCo*rhoJ) - 1

    first = (alpha[1]*betaSr[1]**2+lam*meanCo+lamPri*meanId)/alpha[1] * (tauBar-aK(tauBar)[1])
    #second = 2*lam*tauBar*(muBar-(norMu+rhoJ*meanCo))
    second = 2*lam*(muBar-(norMu+rhoJ*meanCo))

    b1 = first + second
    return b1

def b2(tauBar):
    b2 = betaSr[2]**2*(tauBar-aK(tauBar)[2])
    return b2

In [16]:
# Assuming x_t is x_0 = logS_0
S0=40 #Arbitrary choice
x0 = np.log(S0)
r = 0.0
q = 0.0

def CFlogPrice(z, tau):
    first = 1j*(x0+(r-q)*tau)*z
    second = AkLogPrice(z, tau)+BkLogPrice(z, tau)*sigma0**2
    secondSum = second[1] + second[2]
    CF = np.exp(first + secondSum + capCcoPrice(z, tau) + capCidPrice(z, tau))
    return CF

def CFSV(z1, z2, tau):
    sum1 = AkSV(z1, tau)[1] + BkSV(z1, tau)[1]*sigma0[1]**2
    sum2 = AkSV(z2, tau)[2] + BkSV(z2, tau)[2]*sigma0[2]**2
    CF = np.exp(sum1+sum2+capCcoSV(z1, tau)+capCidSV(z1, tau))
    return CF

In [17]:
def VIXopCall(z, tau, I, K):
    Z1 = -z * aK(tauBar)[1] / tauBar
    Z2 = -z * aK(tauBar)[2] / tauBar

    firstPart = CFSV(Z1, Z2, tau)
    secondPart = np.exp(-1j*z*(b1(tauBar)+b2(tauBar)+I)/tauBar) / ((-1j*z)**1.5)
    thirdPart = 1 - erf((K/100)*(-1j*z)**0.5)
    VIXopInteg = (firstPart*secondPart*thirdPart).real
    
    return VIXopInteg

In [24]:
z = np.linspace(0.5, 4, 1000) * 1j
z = np.array([z])
VIXcall4 = VIXopCall(z, .5, 0.01, 40).flatten()
VIXcall2 = VIXopCall(z, .5, 0.1, 40).flatten()
VIXcall3 = VIXopCall(z, .5, 1, 40).flatten()

imZ = z.imag.flatten()
zOption = imZ
intOption4 = VIXcall4
intOption2 = VIXcall2
intOption3 = VIXcall3
#%matplotlib tk
plt.plot(imZ, VIXcall4)
plt.plot(imZ, VIXcall2)
plt.plot(imZ, VIXcall3)
#plt.yticks(np.arange(0, 1, step=0.05))
plt.xticks(np.arange(0, 4, step=0.25))

plt.legend(['K=10', '40', '70'])
#plt.ylim(0, 1.0)
plt.grid()

In [19]:
#tauBar=30
z = np.linspace(1, 6.8, 1000) * 1j
z = np.array([z])
def VIXfuPricing(z, tau, I):
    Z1 = -z * aK(tauBar)[1] / tauBar
    Z2 = -z * aK(tauBar)[2] / tauBar
    firstPart = CFSV(Z1, Z2, tau)
    secondPart = np.exp(-1j*z*(b1(tauBar)+b2(tauBar)+I)/tauBar) / ((-1j*z)**1.5)

    VIXfuInteg = (firstPart*secondPart).real
    return VIXfuInteg


VIXfu4 = VIXfuPricing(z, 45.0/365, 0.0).flatten()
VIXfu2 = VIXfuPricing(z, .25, 0.0).flatten()
VIXfu3 = VIXfuPricing(z, .5, 0.0).flatten()

imZ = z.imag.flatten()
#%matplotlib tk
plt.plot(imZ, VIXfu4)
plt.plot(imZ, VIXfu2)
plt.plot(imZ, VIXfu3)
plt.yticks(np.arange(0, 1, step=0.05))
plt.xticks(np.arange(0, 6.8, step=0.25))

plt.legend(['45', '3mon', '6mon'])
plt.ylim(0, 1.0)

plt.grid(which='both')

In [20]:
#%matplotlib tk
plt.plot(zOption, intOption4, 'r')
plt.plot(imZ, VIXfu4, 'b--')
plt.plot(zOption, intOption2, 'y')
plt.plot(imZ, VIXfu2, 'm--')
plt.plot(zOption, intOption3, 'k')
plt.plot(imZ, VIXfu3, 'g--')
plt.yticks(np.arange(0, 1, step=0.05))
plt.xticks(np.arange(0, 6.8, step=0.25))
plt.ylim(0, 1.0)
plt.grid(which='both')

In [37]:
u = np.linspace(1, 10, 10000)
u = np.array([u])
r = 0.02
q = 0.0
#Assuming St == S0, so yt = 0
yt = 0
S0 = 1.0
u = 0.0729931217877990394495429
K = 0.8603
tau = 0.238095238095238
I = 0.0
def SPXcall(u, S0, K, I, tau):
    x0 = np.log(S0)
    k = x0 - np.log(K) + (r - q) * tau
    first = np.exp(1j*u*k - 1j*(u - 1j/2)*(x0 - yt + (r - q)*tau))
    second = CFlogPrice(u - 1j/2, tau)
    #second = (9.9994453429116270e-01-3.9708098577177200e-04*1j)
    third = np.exp((-u**2 - 0.25) * I)/(u**2 + 0.25)
    SPXcall = (first*second).real * third
    
    return SPXcall

SPXcall = SPXcall(u, S0, K, I, tau)
print(SPXcall)
#plt.plot(u.flatten(), SPXcall.flatten())
#plt.legend(['SPXcall'])
#plt.yticks(np.arrange())
#plt.grid()

3.906781634474704


In [32]:
u = np.linspace(1, 100, 1000)
u = np.array([u])
CF = CFlogPrice(u-1j/2, .5)
%matplotlib tk

plt.plot(u.flatten(), CF.flatten())
plt.legend(['SPXcall'])
plt.yticks(np.arange(-6.2, 6.2, step=0.25))
plt.grid()

/home/lin/.pyenv/versions/3.7.0/lib/python3.7/site-packages/numpy/core/numeric.py:501: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
pd.DataFrame(CF.flatten()).to_csv(path_or_buf="/home/lin/summerProjectReading/CFlogPrice.csv")

In [ ]:
VIXfu4.shape

In [ ]:
import scipy.integrate.quadrature as integral

In [ ]:
def toInte(z):
    z = np.array([z])
    return VIXfuPricing(z, 45.0/365, 0.0).flatten()

In [62]:
k = 3.0
vbar = 0.1
v0 = 0.08
rho = -0.8
sigma = 0.25
S = 1.0
r = 0.02
tau = 0.714285714285714
tbar = 30/365.0
K = 0.7216

In [63]:
u = np.array([0.0243502926634244325089558, 0.0729931217877990394495429,      
0.1214628192961205544703765, 0.1696444204239928180373136,      
0.2174236437400070841496487, 0.2646871622087674163739642,      
0.3113228719902109561575127, 0.3572201583376681159504426,      
0.4022701579639916036957668, 0.4463660172534640879849477,      
0.4894031457070529574785263, 0.5312794640198945456580139,      
0.5718956462026340342838781, 0.6111553551723932502488530,      
0.6489654712546573398577612, 0.6852363130542332425635584,      
0.7198818501716108268489402, 0.7528199072605318966118638,      
0.7839723589433414076102205, 0.8132653151227975597419233,      
0.8406292962525803627516915, 0.8659993981540928197607834,      
0.8893154459951141058534040, 0.9105221370785028057563807,      
0.9295691721319395758214902, 0.9464113748584028160624815,      
0.9610087996520537189186141, 0.9733268277899109637418535,      
0.9833362538846259569312993, 0.9910133714767443207393824,      
0.9963401167719552793469245, 0.9993050417357721394569056])

In [64]:
iu = 1j * u
var = sigma**2
part1 = -2*k*vbar/var
part2 = np.log(1 - iu*var/(2*k)*(1 - np.exp(-k*tau)))
part3 = iu*np.exp(-k*tau)/(1 - iu*var/(2*k)*(1 - np.exp(-k*tau)))

CF = np.exp(part1*part2+v0*part3)
CF

array([0.99999688+0.00237789j, 0.99997197+0.00712796j,
       0.9999224 +0.01186091j, 0.99984862+0.01656538j,
       0.99975135+0.02123008j, 0.99963151+0.02584383j,
       0.99949023+0.03039557j, 0.99932887+0.03487443j,
       0.99914896+0.03926969j, 0.9989522 +0.04357089j,
       0.99874048+0.04776778j, 0.99851579+0.0518504j ,
       0.99828028+0.05580906j, 0.99803618+0.0596344j ,
       0.9977858 +0.06331739j, 0.99753153+0.06684938j,
       0.99727578+0.07022205j, 0.99702096+0.07342751j,
       0.9967695 +0.07645827j, 0.99652378+0.07930727j,
       0.99628613+0.08196787j, 0.99605879+0.08443392j,
       0.99584393+0.0866997j , 0.99564356+0.08875998j,
       0.99545959+0.09061001j, 0.99529376+0.09224555j,
       0.99514763+0.09366282j, 0.99502259+0.09485859j,
       0.99491981+0.09583012j, 0.99484028+0.09657519j,
       0.99478473+0.09709211j, 0.99475368+0.09737982j])

In [65]:
aTaubar = (1.0 - np.exp(-tbar*k))/k
U = -u*aTaubar/tbar
z = u
u = U
iu = 1j * u
var = sigma**2
part1 = -2*k*vbar/var
part2 = np.log(1 - iu*var/(2*k)*(1 - np.exp(-k*tau)))
part3 = iu*np.exp(-k*tau)/(1 - iu*var/(2*k)*(1 - np.exp(-k*tau)))

CF = np.exp(part1*part2+v0*part3)

bTaubar = vbar*(tbar - aTaubar)
iz = 1j * z
one = np.exp(-iz*bTaubar/tbar)
two = 1.0 - erf((K/100)*(-iz)**0.5)
three = (-iz)**(1.5)

(CF*one*two/three).real


array([-185.64894588,  -35.59954952,  -16.50483434,   -9.9510761 ,
         -6.82529935,   -5.05695757,   -3.94537673,   -3.19471824,
         -2.66079809,   -2.2658463 ,   -1.96461555,   -1.72918571,
         -1.54149872,   -1.38941699,   -1.26451686,   -1.16079204,
         -1.07386013,   -1.00045972,   -0.93812192,   -0.88495018,
         -0.83946925,   -0.80051942,   -0.76718119,   -0.73872074,
         -0.71454996,   -0.69419677,   -0.67728303,   -0.66350778,
         -0.6526348 ,   -0.64448322,   -0.63892051,   -0.63585672])

In [61]:
erf((K/100)*(-iz)**0.5)

array([0.00089844-0.00089844j, 0.00155553-0.00155553j,
       0.00200659-0.00200658j, 0.00237142-0.0023714j ,
       0.00268467-0.00268465j, 0.00296214-0.00296211j,
       0.00321251-0.00321247j, 0.00344118-0.00344114j,
       0.00365173-0.00365168j, 0.00384668-0.00384662j,
       0.00402785-0.00402778j, 0.00419664-0.00419657j,
       0.00435411-0.00435402j, 0.00450108-0.00450099j,
       0.00463823-0.00463813j, 0.00476609-0.00476597j,
       0.00488509-0.00488497j, 0.0049956 -0.00499547j,
       0.00509792-0.00509778j, 0.00519229-0.00519214j,
       0.00527892-0.00527877j, 0.00535799-0.00535783j,
       0.00542964-0.00542947j, 0.005494  -0.00549383j,
       0.00555117-0.00555099j, 0.00560123-0.00560105j,
       0.00564427-0.00564408j, 0.00568033-0.00568013j,
       0.00570946-0.00570926j, 0.0057317 -0.00573151j,
       0.00574709-0.00574689j, 0.00575563-0.00575543j])